Q1

In [1]:
import numpy as np
import math
import operator
from scipy.linalg import svd
import scipy.stats as ss
import matplotlib.pyplot as plt
import scipy.linalg as la

x = np.array([[-2], [-5], [-3], [0], [-8], [-2], [1], [5], [-1], [6]])
y = np.array([[1], [-4], [1], [3], [11], [5], [0], [-1], [-3], [1]])

def lse_1(x,y):
    biasF = np.ones(((len(x)), 1))
    X = np.hstack((biasF, x))

    w = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)

    return w
    

w = lse_1(x,y)

Y = np.zeros(shape=(10,1))

for j in range(len(y)):
    Y[j] = (w[1]*x[j] + w[0])

print("Y_pred: \n", Y)

def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2))

rootmean = rmse(Y, y)
print("RMSE: ", rmse(Y, y))

Y_pred: 
 [[ 1.85394655]
 [ 3.0919826 ]
 [ 2.26662523]
 [ 1.02858919]
 [ 4.33001865]
 [ 1.85394655]
 [ 0.6159105 ]
 [-1.03480423]
 [ 1.44126787]
 [-1.44748291]]
RMSE:  3.7013259176662716


Q2

Q3

In [37]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc
import matplotlib.cm as cm
from collections import Counter
import glob
from matplotlib.image import imread
from enum import Enum
from scipy.linalg import svd
import scipy.stats as ss

# For testing
from sklearn.naive_bayes import GaussianNB


imp_data = np.genfromtxt('spambase.data', delimiter=',')

def train_test_split(data, train_size):
    '''Splitting testing and training data'''

    # Resetting random seed
    np.random.seed(0)

    n = len(data)

    # Rows shuffled
    np.random.shuffle(data)

    # Calculates array index for splitting
    spltIdx = int((2/3)*n)

    # Training-validation data split
    data_train, data_test = data[:spltIdx,:], data[spltIdx:,:]

    # Training data
    x_tr, y_tr = np.hsplit(data_train, [-1])
    # Testing Data
    x_tt, y_tt = np.hsplit(data_test, [-1])

    # Separating class label from data
    class_label_tr = data_train[:, -1].astype(int)
    class_label_tr = class_label_tr.reshape(class_label_tr.shape[0], 1)
    dataset_tr = data_train[:, :-1]

    class_label_tt = data_test[:, -1].astype(int)
    class_label_tt = class_label_tt.reshape(class_label_tt.shape[0], 1)
    dataset_tt = data_test[:, :-1]

    # Filtering features with low std
    dataset_tr = std_filter(dataset_tr, 0.001)
    dataset_tt = std_filter(dataset_tt, 0.001)

    return class_label_tr, dataset_tr, class_label_tt, dataset_tt


def std_filter(data, std_val):
    '''Filters out features with low std'''

    # Standardizing the matrix
    standardized_mat = (data - np.mean(data)) / np.std(data)

    std_mat = np.std(standardized_mat, axis = 0)
    col_num = standardized_mat.shape[1]
    low_idx = []

    for i in range(col_num):
        if std_mat[i] <= std_val:
            low_idx.append(i)

    temp = 0
    for j in low_idx:
        standardized_mat = np.delete(standardized_mat, j - temp, 1)
        temp += 1

    return standardized_mat


def class_sep(data, label):
    '''Separates spam and not spam rows'''

    spIdx_lst = np.where(~label.any(axis=1))[0]
    notIdx_sp_lst = np.where(label.any(axis=1))[0]

    d_list = data.tolist()
    sp_list = []
    not_sp_list = []

    print(data.shape)

    for index in spIdx_lst:
        sp_list += [d_list[index]]

    for index in notIdx_sp_lst:
        not_sp_list += [d_list[index]]

    # print(sp_data)
    sp_data = np.asarray(sp_list)
    not_sp_data = np.asarray(not_sp_list)
    return sp_data, not_sp_data


def nb_fit(x, y):
    n_samples, n_features = x.shape
    print(y)
    classes = np.unique(y)
    print(classes)




def norm_pdf(data, class_idx):
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    numerator = np.exp(- (x-mean)**2 / (2 * var))
    denominator = np.sqrt(2 * np.pi * var)
    return numerator / denominator


class_label_tr, dataset_tr, class_label_tt, dataset_tt = train_test_split(imp_data, 2/3)

sp_data, not_sp_data = class_sep(dataset_tr, class_label_tr)

nb_fit(sp_data, not_sp_data)



# Training data
# x_tr, y_tr = np.hsplit(dataset_tr, [-1])
# Testing Data
# x_tt, y_tt = np.hsplit(dataset_tt, [-1])

(3067, 56)
[[-0.06714524 -0.06606198 -0.05728751 ...  0.02439089  7.17883358
  14.5667205 ]
 [-0.06822851 -0.05934572 -0.06476206 ... -0.04377916  0.07259631
   1.64333473]
 [-0.06822851 -0.06822851 -0.06508704 ... -0.01523505  1.43751384
   2.89992547]
 ...
 [-0.06822851 -0.06042898 -0.04862136 ...  0.13109278  4.18901424
   7.90462306]
 [-0.06822851 -0.06140393 -0.06822851 ... -0.03021664  0.91754526
   1.94664974]
 [-0.06822851 -0.06822851 -0.06822851 ... -0.05197949 -0.0248978
   0.0292656 ]]
[-6.82285127e-02 -6.81960147e-02 -6.81851820e-02 ...  9.91916071e+01
  1.08139400e+02  1.08930185e+02]
